In [684]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [685]:
import re
import pandas as pd
from collections import defaultdict

def get_transaction_count(line):
    pattern = r"transactions = \{([^}]+)\}"
    match = re.search(pattern, line)
    if match:
        return set(re.findall(r"Transaction ([a-fA-F0-9]+)", match.group(1)))
    return set()

def get_timestamp(line):
    pattern = r"^\d+\.\d+"
    match = re.match(pattern, line)
    return float(match.group(0)) if match else None

def get_node_name(line):
    pattern = r"Node ([A-Z0-9]+)"
    match = re.search(pattern, line)
    return match.group(1) if match else None

def count_unique_mempool_transactions(file_path, node_number):
    unique_transactions = set()
    with open(file_path, 'r') as file:
        for line in file:
            if f"Node {node_number}" in line and "from mempool" in line:
                unique_transactions.update(re.findall(r"Transaction ([a-fA-F0-9]+)", line))
    return len(unique_transactions)


def process_log_lines(file_path):
    node_data = defaultdict(lambda: {
        "Timestamp of finalisation": None, 
        "Finalised transactions": set(),  
        "Externalize messages": []
    })
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    for line in lines:
        if 'appended SCPExternalize message to its storage and state' not in line:
            continue
        
        node_name = get_node_name(line)
        timestamp = get_timestamp(line)
        transactions = get_transaction_count(line)
        
        if node_name:
            if node_data[node_name]["Timestamp of finalisation"] is None:
                node_data[node_name]["Timestamp of finalisation"] = timestamp
            node_data[node_name]["Finalised transactions"].update(transactions)
            node_data[node_name]["Externalize messages"].append(line.strip())
    
    df = pd.DataFrame.from_dict(node_data, orient='index')
    df.index.name = "sequence number"
    df = df.reset_index()
    
    df["No. of finalised transactions"] = df["Finalised transactions"].apply(len)
    
    df["total_transactions"] = df["sequence number"].apply(lambda node: count_unique_mempool_transactions(file_path, node))
    
    df["no. of transactions not finalised"] = df["total_transactions"] - df["No. of finalised transactions"]
    
    return df


In [686]:
file_path = 'src/simulator_events_log.txt'

df = process_log_lines(file_path)
df_sorted = df.sort_values(by='Timestamp of finalisation', ascending=True)

ADD INTERLEDGER CHECKS

In [687]:
def calculate_inter_ledger_agreement_time(df):
    df = df.sort_values(by='Timestamp of finalisation')
    time_diffs = df['Timestamp of finalisation'].diff().dropna()
    
    return time_diffs.mean()

avg_time = calculate_inter_ledger_agreement_time(df_sorted)
print(f"Average Inter-Ledger Agreement Time: {avg_time}")

Average Inter-Ledger Agreement Time: 6.043714285714286


In [688]:
final_experiment_df = df_sorted[[
    "sequence number",
    "Timestamp of finalisation",
    "No. of finalised transactions",
    "no. of transactions not finalised"
]]

display(final_experiment_df)

,sequence number,Timestamp of finalisation,No. of finalised transactions,no. of transactions not finalised
0,13,55.41,24,294
1,79,57.42,30,291
2,87,61.68,26,290
3,24,62.28,20,288
4,80,66.68,9,301
5,85,74.46,41,325
6,70,80.97,65,250
7,28,85.06,50,275
8,44,88.22,68,242
9,56,90.76,97,194


In [689]:
avg_difference = (final_experiment_df["no. of transactions not finalised"] - final_experiment_df["No. of finalised transactions"]).mean()

print(f"Average difference: {avg_difference}")

Average difference: 365.0


In [690]:
avg_finalised = final_experiment_df["No. of finalised transactions"].mean()
avg_total = (final_experiment_df["No. of finalised transactions"] + 
             final_experiment_df["no. of transactions not finalised"]).mean()

finalised_percentage = (avg_finalised / avg_total) * 100 if avg_total != 0 else 0

print(f"Percentage of finalised transactions vs total: {finalised_percentage:.2f}%")

Percentage of finalised transactions vs total: 11.49%
